# Lat Long Analysis

This portion of our project uses the schools' latitude and longitude (previously obtained using geocoder) to perform some final cleaning on the VADIR data and then join it to the NYC crime data by location. The code below will:  

* __Load vadir data__ and cleaning it with the functions from cleandata.py  
* Ensure __schools are consistently named__ (we'll use the names from the lat long file and join them using the beds/seds code. This also means that we'll discard records from the schools for which we don't have lat/long)  
* __Fill in missing boroughs__ for records from 2006-2007.  
* __Identify felonies within a 1 mile__ radius of a given school.  
* __Plot correlations__ between school indicents and felonies (by year, by borough, by felony type, by location, by school incident type).

In [1]:
% matplotlib inline
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import vincenty
import cleandata as cd

### Read in and clean the VADIR data from the 2006-2014 school years.

In [2]:
# Raw data for each year
RAW_DATA_DICT = {2006: 'VADIR_2006.xls', 2007: 'VADIR_2007.xls', 2008: 'VADIR_2008.xls', 
                 2009: 'VADIR_2009.xls', 2010: 'VADIR_2010.xls', 2011: 'VADIR_2011.xls', 
                 2012: 'VADIR_2012.xls', 2013: 'VADIR_2013.xls', 2014: 'VADIR_2014.xls'}

# Duplicate name columns in raw files (and their replacements)
DUP_COLS = {'County Name':'County', 'District Name': 'District', 'BEDS CODE': 'BEDS Code', 
            'False Alarm':'Bomb Threat False Alarm',
            'Other Sex offenses': 'Other Sex Offenses', 
            'Use Possession or Sale of Drugs': 'Drug Possession', 
            'Use Possession or Sale of Alcohol': 'Alcohol Possession',
            'Other Disruptive Incidents': 'Other Disruptive Incidents', 
            'Drug Possesion': 'Drug Possession', 'Alcohol Possesion': 'Alcohol Possession', 
            'Other Disruptive': 'Other Disruptive Incidents'}

# Read in raw data and correct duplicate columns
vadir_df = cd.vadir_concat_dfs(RAW_DATA_DICT, DUP_COLS)

... data from VADIR_2006.xls appended. Added 1455 rows for a total of 1455.
... data from VADIR_2007.xls appended. Added 1500 rows for a total of 2955.
... data from VADIR_2008.xls appended. Added 1545 rows for a total of 4500.
... data from VADIR_2009.xls appended. Added 1531 rows for a total of 6031.
... data from VADIR_2010.xls appended. Added 1678 rows for a total of 7709.
... data from VADIR_2011.xls appended. Added 1693 rows for a total of 9402.
... data from VADIR_2012.xls appended. Added 1735 rows for a total of 11137.
... data from VADIR_2013.xls appended. Added 1792 rows for a total of 12929.
... data from VADIR_2014.xls appended. Added 1805 rows for a total of 14734.


In [3]:
# Reorder columns putting demographic information first.
DEMO_COLS = ['School Name', 'School Type', 'School Year', 'BEDS Code',  'County', 
             'District', 'Enrollment', 'Grade Organization', 'Need/Resource Category']
vadir_df = cd.vadir_reorder_columns(vadir_df, DEMO_COLS)

In [4]:
# Create Columns for "Total incidents", "Incidents w/out Weapons" and "Incidents w/ Weapons"
COLUMNS = vadir_df.columns.tolist()
INCIDENT_COLS = [c for c in COLUMNS if c not in DEMO_COLS]
vadir_df = cd.vadir_create_tallies(vadir_df, INCIDENT_COLS)

In [5]:
# Cleaning VADIR data...
# ... consistently name county and school type, fix name capitalization, remove comment rows.
school_df = cd.vadir_clean_concat_df(vadir_df)

In [ ]:
# Take a look
school_df.head()

### Join School Data and Location Data

In [6]:
# read in lat-long file that contains correct names and locations
#... ??? replace this call with function call to geocoder ???
latlon_df = pd.read_csv('SchoolLatLon.csv', index_col=0)

In [ ]:
# take a look
latlon_df.head()

In [7]:
# ensure BEDS and SED are integers so that they'll be recognized as identical
latlon_df["SED CODE"] = latlon_df["SED CODE"].astype(np.int64)
school_df["BEDS Code"] = school_df["BEDS Code"].astype(np.int64)

In [8]:
# join latlong data to school data using the BEDS code
school_df = pd.merge(school_df, latlon_df, left_on=['BEDS Code'],right_on=['SED CODE'], how='left')

In [9]:
# drop the now redundant SED code
school_df.drop(['SED CODE'], axis=1, inplace=True)

In [10]:
# Take a look at the resulting data/missing values
print('... {} unique schools,'.format(len(school_df['BEDS Code'].unique())))
schools_withloc = school_df[school_df['latlon'].notnull()]['BEDS Code'].unique()
schools_missingloc = school_df[school_df['latlon'].isnull()]['BEDS Code'].unique()
print('... of which {} have lat/long'.format(len(schools_withloc)))
print('... and {} are missing lat/long'.format(len(schools_missingloc)))

... 1967 unique schools,
... of which 1807 have lat/long
... and 160 are missing lat/long


### Fix School Names

In [11]:
# The problem:
print('... original dataset has {} unique school names'.format(len(school_df['School Name'].unique())))
print('... but only {} unique BEDS Codes'.format(len(school_df['BEDS Code'].unique())))

... original dataset has 3135 unique school names.
... but only 1967 unique BEDS Codes.


In [12]:
# Helper Function
def fix_case(x):
    """Function to put a school name in the correct case"""
    if not x:
        return x
    elif x[:3] in ['PS ', 'JHS', 'MS ']:
        return x[:3] + x[3:].title()
    else:
        return x.title()

In [13]:
# Fix missing LEGAL NAMES with School Name
school_df['LEGAL NAME'].fillna(school_df['School Name'], inplace=True)

# Fix case and reassign to School Name
school_df['School Name'] = school_df['LEGAL NAME'].apply(fix_case)

# drop the now redundant LEGAL NAME column
school_df.drop(['LEGAL NAME'], axis=1, inplace=True)

In [14]:
# Problem Solved
print('... new dataset has {} unique school names.'.format(len(school_df['School Name'].unique())))
print('... and {} unique BEDS Codes.'.format(len(school_df['BEDS Code'].unique())))

... new dataset has 1959 unique school names.
... and 1967 unique BEDS Codes.


### Fill in missing boroughs

... It turns out this was assigned to Svetlana so I'm going to pause on this item until we talk tomorrow.

In [30]:
# How many BEDS Codes are linked with more than one Borough(County)?
#school_df.groupby('BEDS Code')['County'].apply(lambda x: len(x.unique())).value_counts()

In [31]:
# Lets look at one of these 'multi county schools'
#school_df.groupby('BEDS Code')['County'].apply(lambda x: len(x.unique())).head()

In [32]:
#school_df[school_df['BEDS Code'] == 307500011035]['County'].unique()

In [ ]:
# for each row with nan in borough, id borough of same beds code in another year.

### Function to identify crimes with in a mile of a school.

In [ ]:
# Helper function  
def is_in_radius(school_point, crime_point, radius):
    """
    Function using vincenty package to check distance between school and crime.
    INPUT: (lat,long) tuples for school and crime (in degrees), radius in miles.
    OUTPUT: Boolean
    """
    return vincenty(school_point, crime_point, miles=True) <= radius

In [ ]:
# load NYC dataframe

In [ ]:
# fxn to tally specified types of crimes w/in radius of school
# ... looping is going to be super inefficient, find a way to do this with DP.